In [79]:
import pandas as pd
import numpy as np

In [80]:
pd.read_csv('D:/I/02.Flip Robo Assignments/01.Flip Robo Assignments/12.Ratings Prediction Project/01.Ratings Prediction Project Report Latest/Rating_Prediction_CSV.csv')

stars                                    complete_review
0        3.8  Best sound and best quality light weight very ...
1        3.8  BEST PRICE super good quality very happy 😊😊😊😊 ...
2        4.0  The neckband is good for value for money but t...
3        4.3             Awesome Sound Quality and bass Quality
4        4.2                         Nice quality at this price
...      ...                                                ...
20404    3.9                                               Nice
20405    4.1                                    Good product 👍🏻
20406    4.1                                       Nice product
20407    4.3  Working as expected\nIt suits for BSNL.\nTelep...
20408    4.5                                               good

[20409 rows x 2 columns]

In [81]:
train_data = pd.read_csv('D:/I/02.Flip Robo Assignments/01.Flip Robo Assignments/12.Ratings Prediction Project/01.Ratings Prediction Project Report Latest/Rating_Prediction_CSV.csv')

In [82]:
train_data

stars                                    complete_review
0        3.8  Best sound and best quality light weight very ...
1        3.8  BEST PRICE super good quality very happy 😊😊😊😊 ...
2        4.0  The neckband is good for value for money but t...
3        4.3             Awesome Sound Quality and bass Quality
4        4.2                         Nice quality at this price
...      ...                                                ...
20404    3.9                                               Nice
20405    4.1                                    Good product 👍🏻
20406    4.1                                       Nice product
20407    4.3  Working as expected\nIt suits for BSNL.\nTelep...
20408    4.5                                               good

[20409 rows x 2 columns]

In [83]:
train_data.head()

stars                                    complete_review
0    3.8  Best sound and best quality light weight very ...
1    3.8  BEST PRICE super good quality very happy 😊😊😊😊 ...
2    4.0  The neckband is good for value for money but t...
3    4.3             Awesome Sound Quality and bass Quality
4    4.2                         Nice quality at this price

In [84]:
train_data.shape

(20409, 2)

In [85]:
#3.2 Checking the missing values

In [86]:
train_data.isnull().sum()

stars               0
complete_review    15
dtype: int64

In [87]:
train_data.isnull().sum()*100/train_data.shape[0]

stars              0.000000
complete_review    0.073497
dtype: float64

In [88]:
train_data['complete_review'].fillna(' ',inplace=True)

In [89]:
train_data.isnull().sum()

stars              0
complete_review    0
dtype: int64

In [90]:
#4.Data Prpcessing

In [91]:
# epanding English contraction: https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    #specific
    #phrase = re.sub(r"","",phrase)
    phrase = re.sub(r"”",'"',phrase)
    #phrase = re.sub(r"'","",phrase)
    phrase = re.sub(r"won't","will not",phrase)
    phrase = re.sub(r"can\'t","can not",phrase)
    
    #general
    phrase = re.sub(r"n\'t"," not",phrase)
    phrase = re.sub(r"\'re"," are",phrase)
    phrase = re.sub(r"\'s","s",phrase)
    phrase = re.sub(r"\'d"," would",phrase)
    phrase = re.sub(r"\'ll"," will",phrase)
    phrase = re.sub(r"\'t"," not",phrase)
    phrase = re.sub(r"\'ve"," have",phrase)
    phrase = re.sub(r"\'m"," am",phrase)
    return phrase

In [92]:
train_data['stars'].value_counts()

4.3    3687
4.1    2349
4.2    2163
4.4    1974
4.0    1932
3.9    1905
4.5    1452
3.8    1137
3.6     729
3.7     675
4.6     564
3.4     375
3.3     363
4.7     342
3.2     234
3.5     198
4.8     171
3.0      99
4.9      27
3.1      18
5.0      15
Name: stars, dtype: int64

In [93]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [94]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\COTMAC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [95]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\COTMAC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [96]:
stop_words=stopwords.words('english')

In [97]:
lemmatizer = WordNetLemmatizer()

In [98]:
from tqdm import tqdm
preprocessed_Review = []

In [99]:
#tqdm is for printing the status bar
for sentance in tqdm(train_data['complete_review'].values):
    sent = decontracted(sentance)
    sent = re.sub(r'https?:\/\/.*[\r\n]*','', sent) #remove hyperlinks
    sent = re.sub('A-Za-z]+', ' ', sent) #remove special character, numbers: https://stackoverflow.com/a/5943547/4084039
    sent = ' '.join( e for e in sent.split() if e not in stop_words) # removing stop words
    sent = ' '.join(lemmatizer.lemmatize(e) for e in sent.split()) #Lemmatization
    preprocessed_Review.append(sent.lower().strip())

100%|██████████| 20409/20409 [00:16<00:00, 1230.56it/s]


In [100]:
train_data['complete_review']=preprocessed_Review

In [101]:
#splitting data into train, cv, test

In [102]:
from sklearn.model_selection import train_test_split

In [103]:
#x ==> independant features
#y ==> target

In [104]:
x = train_data.drop(columns ={'stars'})
output=train_data['stars']

In [105]:
train,test,train_output,test_output = train_test_split(x,
                                                       output,
                                                       test_size=0.3,
                                                       stratify = output,
                                                       random_state=0)

train_modified,cv,train_output_modified,cv_output = train_test_split(train,
                                                                     train_output,
                                                                     test_size=0.3,
                                                                     stratify=train_output,
                                                                     random_state=0)

In [106]:
train.shape,cv.shape,test.shape

((14286, 1), (4286, 1), (6123, 1))

In [107]:
train_output.shape,cv_output.shape,test_output.shape

((14286,), (4286,), (6123,))

In [108]:
#6.Data Encoding
#TFIDF Vectorization

In [109]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [110]:
Review_tfidf_Vectorizer = TfidfVectorizer(min_df=5)

In [111]:
train_Review_tfidf=Review_tfidf_Vectorizer.fit_transform(train['complete_review'].values)

In [112]:
cv_Review_tfidf=Review_tfidf_Vectorizer.transform(cv['complete_review'].values)
test_Review_tfidf=Review_tfidf_Vectorizer.transform(test['complete_review'].values)

In [113]:
#solving tfidf vectorizer
import pickle
with open("Review_tfidf_Vectorizer.pickle","wb") as fp:
    pickle.dump(Review_tfidf_Vectorizer,fp,protocol=pickle.HIGHEST_PROTOCOL)

In [114]:
Review_tfidf_Vectorizer.get_feature_names()[:10]

['00', '000', '01', '02', '03', '06', '08', '09', '10', '100']

In [115]:
# TFIDF and Linear SVM

In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [117]:
tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,3), analyzer ='char')

In [118]:
x = tfidf.fit_transform(train_data['complete_review'])
y = train_data['stars']

In [119]:
x.shape, y.shape

((20409, 20000), (20409,))

In [125]:
from sklearn import preprocessing
from sklearn import utils

#convert y values to categorical values
lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y)

#view transformed values
print(y_transformed)

[ 8  8 10 ... 11 13 15]


In [138]:
x_train, x_test, y_train, y_test = train_test_split(x,y_transformed,test_size = 0.2, random_state = 0)

In [139]:
x_train.shape

(16327, 20000)

In [140]:
#fit logistic regression model
classifier = LogisticRegression()
classifier.fit(x, y_transformed)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [141]:
clf = LinearSVC(C = 10, class_weight = 'balanced')
clf.fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(C=10, class_weight='balanced')

In [142]:
y_pred = clf.predict(x_test)

In [143]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.38      0.69      0.49        26
           1       0.07      1.00      0.12         4
           2       0.34      0.84      0.48        50
           3       0.57      0.73      0.64        75
           4       0.50      0.81      0.62        84
           5       0.35      0.97      0.51        31
           6       0.43      0.63      0.51       139
           7       0.44      0.59      0.51       130
           8       0.53      0.56      0.54       235
           9       0.58      0.47      0.52       391
          10       0.59      0.45      0.51       380
          11       0.62      0.43      0.51       468
          12       0.62      0.56      0.59       427
          13       0.67      0.41      0.51       736
          14       0.55      0.44      0.49       413
          15       0.40      0.42      0.41       279
          16       0.41      0.75      0.53       107
          17       0.29    

In [144]:
clf = LinearSVC(C = 20, class_weight = 'balanced')
clf.fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(C=20, class_weight='balanced')

In [145]:
y_pred = clf.predict(x_test)

In [146]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.38      0.69      0.49        26
           1       0.07      1.00      0.12         4
           2       0.34      0.84      0.48        50
           3       0.57      0.72      0.64        75
           4       0.49      0.82      0.62        84
           5       0.35      0.97      0.51        31
           6       0.44      0.61      0.51       139
           7       0.45      0.59      0.51       130
           8       0.52      0.54      0.53       235
           9       0.57      0.47      0.52       391
          10       0.58      0.44      0.50       380
          11       0.62      0.43      0.50       468
          12       0.61      0.55      0.58       427
          13       0.66      0.41      0.51       736
          14       0.55      0.44      0.49       413
          15       0.40      0.43      0.42       279
          16       0.42      0.75      0.54       107
          17       0.30    

In [147]:
tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,5), analyzer ='char')

In [149]:
x = tfidf.fit_transform(train_data['complete_review'])
y = train_data['stars']

In [150]:
x.shape, y.shape

((20409, 20000), (20409,))

In [152]:
x_train, x_test, y_train, y_test = train_test_split(x,y_transformed,test_size = 0.2, random_state = 0)

In [153]:
x_train.shape

(16327, 20000)

In [154]:
clf = LinearSVC(C = 20, class_weight = 'balanced')
clf.fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(C=20, class_weight='balanced')

In [155]:
y_pred = clf.predict(x_test)

In [156]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.38      0.69      0.49        26
           1       0.07      1.00      0.12         4
           2       0.33      0.84      0.48        50
           3       0.57      0.72      0.64        75
           4       0.49      0.82      0.62        84
           5       0.33      0.90      0.48        31
           6       0.45      0.58      0.50       139
           7       0.44      0.59      0.50       130
           8       0.53      0.52      0.53       235
           9       0.56      0.47      0.51       391
          10       0.57      0.46      0.51       380
          11       0.62      0.43      0.50       468
          12       0.61      0.56      0.58       427
          13       0.65      0.42      0.51       736
          14       0.56      0.43      0.48       413
          15       0.41      0.44      0.43       279
          16       0.42      0.75      0.54       107
          17       0.29    

In [157]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,roc_curve,roc_auc_score,auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [158]:
# LogisticRegression
LG = LogisticRegression(C=1, max_iter = 3000)

LG.fit(x_train, y_train)

y_pred_train = LG.predict(x_train)
print('Training accuracy is {}'.format(accuracy_score(y_train, y_pred_train)))
y_pred_test = LG.predict(x_test)
print('Test accuracy is {}'.format(accuracy_score(y_test,y_pred_test)))
print(confusion_matrix(y_test,y_pred_test))
print(classification_report(y_test,y_pred_test))

Training accuracy is 0.4746126048876095
Test accuracy is 0.3417442430181284
[[  0   0   0   0   0   0   0   0   0   0   2   8   3  11   2   0   0   0
    0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   1   0   3   0   0   0   0
    0   0   0]
 [  0   0   1   0   0   0   0   0   0   1   5   7  10  24   0   2   0   0
    0   0   0]
 [  0   0   0   0   0   0   1   0   2   2   3  14  10  38   2   3   0   0
    0   0   0]
 [  0   0   0   0   1   0   0   0   2   4   2  10  15  45   5   0   0   0
    0   0   0]
 [  0   0   0   0   0   0   0   0   0   2   4   4   4  13   4   0   0   0
    0   0   0]
 [  0   0   0   0   0   0  10   2   4   4   8  13  23  67   6   1   0   1
    0   0   0]
 [  0   0   0   0   1   0   2   8   1   7  10  13  18  51  12   7   0   0
    0   0   0]
 [  0   0   0   0   0   0   0   0  26  18  16  34  19  98  18   6   0   0
    0   0   0]
 [  0   0   0   0   1   0   2   2  10 126  21  26  23 139  31  10   0   0
    0   0   0]
 [  0   0   0   0   0   0   3   0 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [159]:
# DecisionTreeClassifier
DT = DecisionTreeClassifier()

DT.fit(x_train, y_train)
y_pred_train = DT.predict(x_train)
print('Training accuracy is {}'.format(accuracy_score(y_train, y_pred_train)))
y_pred_test = DT.predict(x_test)
print('Test accuracy is {}'.format(accuracy_score(y_test,y_pred_test)))
print(confusion_matrix(y_test,y_pred_test))
print(classification_report(y_test,y_pred_test))

Training accuracy is 0.6528449807068046
Test accuracy is 0.5122488975992161
[[ 16   0   0   0   0   0   0   4   0   0   1   0   2   0   0   0   3   0
    0   0   0]
 [  0   3   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0
    0   0   0]
 [  1   0  28   0   1   0   2   4   2   2   4   2   0   4   0   0   0   0
    0   0   0]
 [  1   0   1  40   2   0   2   1   0   4   3   3   4   9   4   1   0   0
    0   0   0]
 [  1   0   2   0  45   0   1   2   1   3   2   2   5  17   1   2   0   0
    0   0   0]
 [  0   0   0   0   0  19   0   0   0   3   1   0   2   4   0   2   0   0
    0   0   0]
 [  0   0   2   5   1   0  68   3   5   7  10  11  12  12   2   0   0   1
    0   0   0]
 [  0   0   2   1   2   0   8  62   3  10   9   3  10  13   4   3   0   0
    0   0   0]
 [  1   0   0   0   1   1   5   3 118  13   9  14  15  34  12   5   2   2
    0   0   0]
 [  0   0   2   4   6   0  13  13   8 210  20  16  17  54  14   8   1   4
    1   0   0]
 [  1   0   2   6   1   2  11   4 

In [160]:
#RandomForestClassifier
RF = RandomForestClassifier()

RF.fit(x_train, y_train)
y_pred_train = RF.predict(x_train)
print('Training accuracy is {}'.format(accuracy_score(y_train, y_pred_train)))
y_pred_test = RF.predict(x_test)
print('Test accuracy is {}'.format(accuracy_score(y_test,y_pred_test)))
print(confusion_matrix(y_test,y_pred_test))
print(classification_report(y_test,y_pred_test))

Training accuracy is 0.6527224842285785
Test accuracy is 0.5120039196472318
[[ 14   0   1   0   0   0   0   0   0   0   0   5   2   4   0   0   0   0
    0   0   0]
 [  0   3   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0
    0   0   0]
 [  0   0  27   0   1   0   1   3   0   1   4   3   1   7   0   2   0   0
    0   0   0]
 [  1   0   1  34   2   0   3   0   0   4   2   5   4  10   6   1   0   0
    2   0   0]
 [  1   0   2   0  40   0   1   0   1   5   2   2   6  19   2   2   0   1
    0   0   0]
 [  0   0   0   0   0  19   0   0   0   3   1   0   2   4   0   2   0   0
    0   0   0]
 [  0   0   2   1   0   0  66   4   4   7   7   8  14  21   2   2   0   1
    0   0   0]
 [  0   0   2   0   2   0   6  57   1  10   7   6  10  17   6   4   0   2
    0   0   0]
 [  0   0   0   0   0   1   2   1 113   7  11  15  14  44  13   7   2   5
    0   0   0]
 [  0   0   2   3   4   0  14   8  10 197  21  18  22  55  19  11   1   4
    2   0   0]
 [  0   0   3   4   0   2  10   3 

In [161]:
#AdaBoostClassifier
ada=AdaBoostClassifier(n_estimators=100)
ada.fit(x_train, y_train)
y_pred_train = ada.predict(x_train)
print('Training accuracy is {}'.format(accuracy_score(y_train, y_pred_train)))
y_pred_test = ada.predict(x_test)
print('Test accuracy is {}'.format(accuracy_score(y_test,y_pred_test)))
print(confusion_matrix(y_test,y_pred_test))
print(classification_report(y_test,y_pred_test))

Training accuracy is 0.17939609236234458
Test accuracy is 0.1798138167564919
[[  0   0   0   0   0   0   0   0   0   0   0   0   0  26   0   0   0   0
    0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   3   0   0   0   0   0   0
    1   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   2   0  48   0   0   0   0
    0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0  75   0   0   0   0
    0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   2   0  82   0   0   0   0
    0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0  30   0   0   0   0
    1   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0 137   0   0   0   0
    2   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0 129   0   0   0   0
    1   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   2   0 233   0   0   0   0
    0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   2   0 383   0   0   0   0
    6   0   0]
 [  0   0   0   0   0   0   0   0

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [2]:
#As compare multiple model Logistic Regression is better accuracy 0.63.

In [163]:
import pickle
filename ='Rating_Prediction_Latest'
pickle.dump(LG,open(filename,'wb'))

In [72]:
#Conclusion

In [164]:
loaded_model=pickle.load(open('Rating_Prediction_Latest','rb'))
result=loaded_model.score(x_test,y_test)
print(result)

0.3417442430181284


In [165]:
conclusion = pd.DataFrame([loaded_model.predict(x_test)[:],y_pred_test[:]],index=["predicted","origional"])

In [166]:
conclusion

0     1     2     3     4     5     6     7     8     9     ...  \
predicted    13    13    11    12    13    15    13    13    13    13  ...   
origional    13    13    13    13    13    13    13    13    13    13  ...   

           4072  4073  4074  4075  4076  4077  4078  4079  4080  4081  
predicted     9    13    13    12    13    13    13    13     9    12  
origional    13    13    13    13    13    13    13    13    13    13  

[2 rows x 4082 columns]